In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Celiac_Disease/GSE72625'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gastrointestinal symptoms and pathology in patients with Common variable immunodeficiency"
!Series_summary	"Based on the findings of increased IEL in duodenal biopsies in CVID, an overlap with celiac disease has been suggested. In the present study, increased IEL, in particular in the pars descendens of the duodenum, was one of the most frequent histopathological finding. We therefore examined the gene expression profile in pars descendens of duodenum in CVID patients with increased IEL (n=12, IEL mean 34 [range 22-56] IEL/100 EC), CVID with normal levels of IEL (n=8), celiac disease (n=10, Marsh grade 3a or above) and healthy controls (n=17) by gene expression microarray"
!Series_overall_design	"GI histopathological findings in 53 CVID patients that underwent upper and lower endoscopic examination were addressed. For the microarray analysis 20 CVID (8 CVID_normal and 12 CVID with incresed IEL), 10 patients with celiac diseases and 17 healthy cont

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check gene expression data availability
is_gene_available = True  # The dataset mentions gene expression microarray

# Determine variable availability
trait_row = 0  # Disease state information is available at this key
age_row = None  # No information about age in sample characteristics
gender_row = None  # No information about gender in sample characteristics

# Define data conversion functions

def convert_trait(value):
    # Assuming the values are 'celiac disease', 'CVID with increased intraepithelial lymphocytes', 
    # 'CVID without increased intraepithelial lymphocytes', and 'healthy controls'
    mapping = {
        'celiac disease': 1,
        'CVID with increased intraepithelial lymphocytes': 0,
        'CVID without increased intraepithelial lymphocytes': 0,
        'healthy controls': 0
    }
    return mapping.get(value.split(': ')[1], None)

# Age and gender conversion functions are not needed as the data is not available
convert_age = None 
convert_gender = None

# Save cohort information
save_cohort_info('GSE72625', './preprocessed/Celiac_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Celiac_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Celiac_Disease/trait_data/GSE72625.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1866896': [0], 'GSM1866897': [0], 'GSM1866898': [0], 'GSM1866899': [0], 'GSM1866900': [0], 'GSM1866901': [0], 'GSM1866902': [0], 'GSM1866903': [0], 'GSM1866904': [0], 'GSM1866905': [0], 'GSM1866906': [0], 'GSM1866907': [0], 'GSM1866908': [0], 'GSM1866909': [0], 'GSM1866910': [0], 'GSM1866911': [0], 'GSM1866912': [0], 'GSM1866913': [0], 'GSM1866914': [0], 'GSM1866915': [0], 'GSM1866916': [1], 'GSM1866917': [1], 'GSM1866918': [1], 'GSM1866919': [1], 'GSM1866920': [1], 'GSM1866921': [1], 'GSM1866922': [1], 'GSM1866923': [1], 'GSM1866924': [1], 'GSM1866925': [1], 'GSM1866926': [0], 'GSM1866927': [0], 'GSM1866928': [0], 'GSM1866929': [0], 'GSM1866930': [0], 'GSM1866931': [0], 'GSM1866932': [0], 'GSM1866933': [0], 'GSM1866934': [0], 'GSM1866935': [0], 'GSM1866936': [0], 'GSM1866937': [0], 'GSM1866938': [0], 'GSM1866939': [0], 'GSM1866940': [0], 'GSM1866941': [0], 'GSM1866942': [0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which keys to use for mapping identifiers to gene symbols.
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the get_gene_mapping function.
mapping_df = get_gene_mapping(gene_annotation, prob_col=identifier_key, gene_col=gene_symbol_key)

# 3. Apply the mapping with the apply_gene_mapping function, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Celiac_Disease/gene_data/GSE72625.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Celiac_Disease')

# 4. Save the cohort information.
save_cohort_info('GSE72625', './preprocessed/Celiac_Disease/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Celiac_Disease/GSE72625.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Celiac_Disease', the least common label is '1.0' with 10 occurrences. This represents 21.28% of the dataset.
The distribution of the feature 'Celiac_Disease' in this dataset is fine.

